In [8]:
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Define Dimensions
n_x = 2
n_theta = 6

In [10]:
# Initialise parameters
P = np.eye(n_x)
R = 0.1 * np.eye(n_x)
Q = 0.1 * np.eye(n_x)
Upsilon = np.zeros((n_x, n_theta))
S = np.eye(n_theta)

llama = 0.995
a = 0.999

x_hat = np.zeros((n_x,1))
theta_hat = np.zeros((n_theta,1))

# Initial state
x_true = np.array([[0.0], [0.0]])  # Initial position and velocity

In [11]:
# System parameters (unknown, to be estimated)
R_true = 0.75  # Spring constant
K_true = 0.04   # Damping coefficient
L_true = 0.0015  # Mass
J_true = 0.0002
f_true = 0.003

In [12]:
def V_k(t):
    return 1 if np.cos(t) > 0 else -1

In [13]:
ts = 5
dt = 0.001
N = int(ts/dt)
x_estimates = []
x_trues = []
theta_estimates = []

In [14]:
for k in range(N):
    Vk = V_k(k * dt)
    Phi_k = np.array([
        [x_true[0, 0], x_true[1, 0], Vk, 0, 0, 0],
        [0, 0, 0, x_true[0, 0], x_true[1, 0], Vk]
    ])

    # Compute gains
    P = P + Q
    Sigma = P + R
    K = P @ np.linalg.inv(Sigma)
    P = (np.eye(n_x) - K) @ P


    Omega = Upsilon + Phi_k
    Upsilon = (np.eye(n_x) - K) @ Upsilon + (np.eye(n_x) - K) @ Phi_k
    Lambda = np.linalg.inv(llama * Sigma + Omega @ S @ Omega.T)
    Pi = S @ Omega.T @ Lambda
    S = (1 / llama) * S - (1/ llama) * S @ Omega.T @ Lambda @ Omega @ S

    # Estimate state and parameters
    y_k = x_true  # Assume perfect measurements (no noise)
    theta_previous = theta_hat.copy()
    innovation = y_k - (x_hat + Phi_k @ theta_previous)
    theta_hat = theta_previous + Pi @ innovation
    x_hat = x_hat + Phi_k @ theta_previous + K @ innovation + Upsilon @ (theta_hat - theta_previous)
    # theta_hat = theta_hat + theta_update
    
    # Update Covariance matrices
    Q = a * Q + (1 - a) * (K @ (innovation @ innovation.T) @ K.T)
    R = a * R + (1 - a) * (innovation @ innovation.T + P)

    # Store results
    x_estimates.append(x_hat.flatten())
    theta_estimates.append(theta_hat.flatten())

    # True system update
    x_true = x_true + np.array([
        [-R_true/L_true, -K_true/L_true],
        [-K_true/J_true, -f_true/J_true]
    ]) @ x_true + np.array([[1/L_true], [0]]) * Vk

    x_trues.append(x_true.flatten())

/tmp/ipykernel_27264/1346803111.py:17: RuntimeWarning: overflow encountered in matmul
  Lambda = np.linalg.inv(llama * Sigma + Omega @ S @ Omega.T)
/tmp/ipykernel_27264/1346803111.py:17: RuntimeWarning: invalid value encountered in matmul
  Lambda = np.linalg.inv(llama * Sigma + Omega @ S @ Omega.T)
/tmp/ipykernel_27264/1346803111.py:38: RuntimeWarning: overflow encountered in matmul
  x_true = x_true + np.array([
/tmp/ipykernel_27264/1346803111.py:38: RuntimeWarning: invalid value encountered in add
  x_true = x_true + np.array([
